# 20231017_Lung_Xenium_CDH1+

これまでに3種類のdensityの算出方法を試してきた。
1. 全視野の面積に対するXX遺伝子のmRNA数のdensity
2. 全視野の面積に対するXX遺伝子陽性細胞数のdensity
3. 各細胞の面積に対するXX遺伝子のmRNA数のdensity

この中で、Fig①②では1,2が使われており、hot-cold解析では3が使われている。
ひとまず、今回のpre-post解析はhot-coldの方法3で行うこととした。

In [29]:
import stlearn as st
import scanpy as sc
import numpy as np
import pandas as pd
import squidpy as sq
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
# path to Xenium data folder of Lung panel
path_ls = [
    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__1_NSCLC_Case1_Pre__20230821__075702/", #1
    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/output-XETG00053__0003973__SQUAT-Case1__20230502__074131/", 

    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__5_NCLC_Case_2__20230821__075702/", #2 post

    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__6_NSCLC_Cas3_2_Pre__20230821__075702/", #3 pre

    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__2_NSCLC_Case7_Pre__20230821__075702/", #7
    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/output-XETG00053__0003973__SQUAT-Case7__20230502__074131/", 

    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__7_NSCLC_Case_8__20230821__075702/", #8 post

    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__3_NSCLC_Case_11_Pre__20230821__075702/", #11
    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/output-XETG00053__0003973__CRT-Case11__20230502__074131/",

    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/20230821__074259__2_019/Xenium_Outputs/output-XETG00053__0006672__4_NSCLC_Case_12_Pre__20230821__075702/", # 12
    "/Volumes/Project/home/sakai/scRNA/20230518_Xenium/DATA/Xenium_Outputs/output-XETG00053__0003973__CRT-Case12__20230502__074131/" 
]

In [3]:
# set labels
case_ls = ["Case1", "Case1", "Case2", "Case3", "Case7", "Case7", "Case8", "Case11", "Case11", "Case12", "Case12"]
prepost_ls = ["Pre", "Post", "Post", "Pre", "Pre", "Post", "Post", "Pre", "Post", "Pre", "Post"]

## 各細胞の面積に対するXX遺伝子のmRNA数のdensity

In [60]:
# load data of lung panel
df_tumor1_dens_res = pd.DataFrame()
df_tumor2_dens_res = pd.DataFrame()

for i, s, j in zip(path_ls, case_ls, prepost_ls):

    # load Xenium feature matrix
    adata_xenium = st.ReadXenium(
        feature_cell_matrix_file=f"{i}cell_feature_matrix.h5",
        cell_summary_file=f"{i}cells.csv.gz",
        library_id="Xenium_case",
        image_path="/Volumes/Project/home/sakai/sample.jpg",
        scale=1,
        spot_diameter_fullres=15 # Recommend
    )

    # load cell area
    df_cell = pd.read_csv(
        f"{i}cells.csv.gz", 
        index_col="cell_id"
    )
    
    # count table
    df_xenium = adata_xenium.to_df()

    # extract CDH1+ cells
    df_tumor1 = df_xenium[df_xenium["CDH1"] >= 1]
    df_tumor2 = df_xenium[df_xenium["CDH1"] >= 2]

    # calculate density per cell
    df_tumor1_dens = (df_tumor1.T / df_cell.loc[df_tumor1.index, "cell_area"]).T
    df_tumor2_dens = (df_tumor2.T / df_cell.loc[df_tumor2.index, "cell_area"]).T

    # annotate labels
    df_tumor1_dens["Case"] = s
    df_tumor1_dens["Prepost"] = j
    df_tumor2_dens["Case"] = s
    df_tumor2_dens["Prepost"] = j

    # concat
    df_tumor1_dens_res = pd.concat(
        [df_tumor1_dens_res, df_tumor1_dens], axis=0
    )
    df_tumor2_dens_res = pd.concat(
        [df_tumor2_dens_res, df_tumor2_dens], axis=0
    )


Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!


In [173]:
def calculate_q(p_seq):
    p_arr   = np.asarray(p_seq)
    N       = len(p_arr)
    idx_arr = np.argsort(p_arr)
    q_arr   = p_arr[idx_arr] * N / (np.arange(N) + 1)
    q_arr   = np.minimum.accumulate(q_arr[::-1])[::-1]
    q_arr[idx_arr] = q_arr.copy()
    return q_arr

from numpy import std, mean, sqrt

#correct if the population S.D. is expected to be equal for the two groups.
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)

# plot
sns.set('talk', 'whitegrid', 'dark', font_scale=0.8,
        rc={"lines.linewidth": 2, 'grid.linestyle': '--'})

def plot_effect_bar(df, name):
    
    if len(df) == 0:
        return "no significant feature"
    
    # 正をLTSにしたいので、effect sizeの向きを逆転させる
    #df["effect"] = df["effect"] * -1

    # バープロットを作成
    # データを用意
    #sns.set(font_scale=1.2)
    #sns.set_style("white")
    #sns.set_style("ticks")
    fig, ax = plt.subplots(figsize=(4, 20))
    bars = ax.barh(df.index, df["effect"])

    # 軸の設定
    ax.set_xlim(-max(abs(df["effect"]))-0.5, max(abs(df["effect"]))+0.5)
    ax.axvline(x=0, color='black', linewidth=0.5)

    # ラベルを描写
    for i, d, s in zip(df.index, df["effect"], df["q-value"]):

        # テキストアノテーション
        if d < 0:
            ax.text(0.03, i, i, ha='left', va='baseline')
        else:
            ax.text(-0.03, i, i, ha='right', va='baseline')

        # バープロットの色分け
        if d > 0:
            bars[df.index.tolist().index(i)].set_color('b')
        else:
            bars[df.index.tolist().index(i)].set_color('r')


    plt.tick_params(labelleft=False)
    plt.grid(axis="y")
    plt.xlim(-2.3, 2.3)
    sns.despine(left=True)


    plt.tick_params(labelsize=12, left=False)
    ax.set_xlabel("Pre←  Effect size  →Post", fontsize=15)
    #ax.set_xlabel(None)
    fig.tight_layout()
    plt.tight_layout()
    plt.savefig(f"{name}.png", dpi=500)

def compare_prepost(df, name):
        
    # tumor 1+
    res_ls = []
    res_effect_ls = []
    
    for i in df.columns[:-2]:
        A = df.groupby("Prepost").get_group("Pre")[i]
        B = df.groupby("Prepost").get_group("Post")[i]
        
        #res_ls += [stats.mannwhitneyu(A,B,alternative='two-sided').pvalue]
        res_ls += [stats.ttest_ind(A, B, equal_var=False).pvalue]
        res_effect_ls += [cohen_d(B,A)]
    
    #df_res = pd.DataFrame(calculate_q(res_ls), index=df_hot_cold.index)
    df_res = pd.DataFrame([res_ls, res_effect_ls], columns=df.columns[:-2]).dropna().T
    df_res[0] = calculate_q(df_res[0])
    df_res.index.name = ""
    df_res.columns = ["q-value", "effect"]
    df_res.to_csv(f"{name}.txt", sep="\t")
    
    df_res_tumor = df_res[df_res["q-value"] < 0.05].sort_values("effect")
    # 多すぎ
    
    df_res_tumor = df_res_tumor[abs(df_res_tumor["effect"]) > 0.5]
    df_res_tumor.columns = ["q-value", "effect"]

    return df_res_tumor

In [ ]:
# 1+ all pair
df_res_tumor1 = compare_prepost(df_tumor1_dens_res, "tumor1_all")

plot_effect_bar(df_res_tumor1, "tumor1_effect")

# case7
df_res_tumor1_case7 = compare_prepost(
    df_tumor1_dens_res.groupby("Case").get_group("Case7"), "tumor1_case7"
)
plot_effect_bar(df_res_tumor1_case7, "tumor1_case7_effect")

# case1
df_res_tumor1_case1 = compare_prepost(
    df_tumor1_dens_res.groupby("Case").get_group("Case1"), "tumor1_case1"
)

plot_effect_bar(df_res_tumor1_case1, "tumor1_case1_effect")

df_tumor1_dens_res.to_csv("20231017_CDH1_positive_cell_RNA_density_per_cell1.txt", sep="\t")

In [ ]:
# 2+ all pair
df_res_tumor2 = compare_prepost(df_tumor2_dens_res, "tumor2_all")
plot_effect_bar(df_res_tumor2, "tumor2_effect")

# case7
df_res_tumor2_case7 = compare_prepost(
    df_tumor2_dens_res.groupby("Case").get_group("Case7"), "tumor2_case7"
)

plot_effect_bar(df_res_tumor2_case7, "tumor2_case7_effect")

# case1
df_res_tumor2_case1 = compare_prepost(
    df_tumor2_dens_res.groupby("Case").get_group("Case1"), "tumor2_case1"
)

plot_effect_bar(df_res_tumor2_case1, "tumor2_case1_effect")

df_tumor2_dens_res.to_csv("20231017_CDH1_positive_cell_RNA_density_per_cell2.txt", sep="\t")

## 全視野の面積に対するXX遺伝子のmRNA数のdensity

In [48]:
# load data of lung panel
df_tumor1_dens_res = pd.DataFrame()
df_tumor2_dens_res = pd.DataFrame()

for i, s, j in zip(path_ls, case_ls, prepost_ls):

    # load Xenium feature matrix
    adata_xenium = st.ReadXenium(
        feature_cell_matrix_file=f"{i}cell_feature_matrix.h5",
        cell_summary_file=f"{i}cells.csv.gz",
        library_id="Xenium_case",
        image_path="/Volumes/Project/home/sakai/sample.jpg",
        scale=1,
        spot_diameter_fullres=15 # Recommend
    )

    # load cell area
    df_cell = pd.read_csv(
        f"{i}cells.csv.gz", 
        index_col="cell_id"
    )
    
    # count table
    df_xenium = adata_xenium.to_df()

    # extract CDH1+ cells
    df_tumor1 = df_xenium[df_xenium["CDH1"] >= 1]
    df_tumor2 = df_xenium[df_xenium["CDH1"] >= 2]

    # calculate density per cell
    df_tumor1_dens = pd.DataFrame(
        (df_tumor1.sum() / df_cell.loc[df_tumor1.index, "cell_area"].sum())
    ).T
    df_tumor2_dens = pd.DataFrame(
        (df_tumor2.sum() / df_cell.loc[df_tumor2.index, "cell_area"].sum())
    ).T

    # annotate labels
    df_tumor1_dens["Case"] = s
    df_tumor1_dens["Prepost"] = j
    df_tumor2_dens["Case"] = s
    df_tumor2_dens["Prepost"] = j

    # concat
    df_tumor1_dens_res = pd.concat(
        [df_tumor1_dens_res, df_tumor1_dens], axis=0
    )
    df_tumor2_dens_res = pd.concat(
        [df_tumor2_dens_res, df_tumor2_dens], axis=0
    )


Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!
Added tissue image to the object!


In [50]:
def calculate_q(p_seq):
    p_arr   = np.asarray(p_seq)
    N       = len(p_arr)
    idx_arr = np.argsort(p_arr)
    q_arr   = p_arr[idx_arr] * N / (np.arange(N) + 1)
    q_arr   = np.minimum.accumulate(q_arr[::-1])[::-1]
    q_arr[idx_arr] = q_arr.copy()
    return q_arr

res_ls = []

for i in df_tumor1_dens_res.columns[:-2]:
    A = df_tumor1_dens_res.groupby("Prepost").get_group("Pre")[i]
    B = df_tumor1_dens_res.groupby("Prepost").get_group("Post")[i]
    
    #res_ls += [stats.mannwhitneyu(A,B,alternative='two-sided').pvalue]
    res_ls += [stats.ttest_ind(A, B, equal_var=False).pvalue]

#df_res = pd.DataFrame(calculate_q(res_ls), index=df_hot_cold.index)
df_res = pd.DataFrame(res_ls, index=df_tumor1_dens_res.columns[:-2]).dropna()
df_res[0] = calculate_q(df_res[0])
df_res.index.name = ""

In [56]:
df_res.sort_values(0)
# 有意差がつかない

,0
,
ZEB1,0.236698
CD86,0.236698
C1QC,0.236698
CLDN25,0.236698
FRZB,0.236698
...,...
CD44,0.946036
FABP4,0.955831
PDCD1,0.961851


In [57]:
def calculate_q(p_seq):
    p_arr   = np.asarray(p_seq)
    N       = len(p_arr)
    idx_arr = np.argsort(p_arr)
    q_arr   = p_arr[idx_arr] * N / (np.arange(N) + 1)
    q_arr   = np.minimum.accumulate(q_arr[::-1])[::-1]
    q_arr[idx_arr] = q_arr.copy()
    return q_arr

res_ls = []

for i in df_tumor2_dens_res.columns[:-2]:
    A = df_tumor2_dens_res.groupby("Prepost").get_group("Pre")[i]
    B = df_tumor2_dens_res.groupby("Prepost").get_group("Post")[i]
    
    #res_ls += [stats.mannwhitneyu(A,B,alternative='two-sided').pvalue]
    res_ls += [stats.ttest_ind(A, B, equal_var=False).pvalue]

#df_res = pd.DataFrame(calculate_q(res_ls), index=df_hot_cold.index)
df_res = pd.DataFrame(res_ls, index=df_tumor1_dens_res.columns[:-2]).dropna()
df_res[0] = calculate_q(df_res[0])
df_res.index.name = ""

In [59]:
df_res.sort_values(0)

,0
,
ACE,0.445235
LGI4,0.445235
LAT,0.445235
JUND,0.445235
ITGBL1,0.445235
...,...
CRELD2,0.978295
TBX21,0.979401
WNT2,0.979401
